In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import causalimpact
import os
import random
import copy
import warnings
import tensorflow as tf
import keras
import tensorflow_probability as tfp
import time
from causalimpact import CausalImpact

In [ ]:
np.random.seed(10)
x = np.linspace(1, 200, 200)
y = np.linspace(30, 100, 100)
oney = np.ones(100) * 30
yconc = np.concatenate([oney, y])
ycontroll = np.ones(200) * 20
noise = np.random.uniform(-5, 5, size=200)
yfinal = np.add(yconc, noise)
ycontrfin = np.add(ycontroll, noise)
cm = 1 / 2.54
fig3, ax3 = plt.subplots(figsize=(80 * cm, 40 * cm))
ax3.plot(yfinal, linewidth=2.5, color='red', label='Treated group')
ax3.plot(ycontrfin, linewidth=2.5, color='blue', label='Control group')
plt.legend(fontsize=30, loc="best")
ax3.set_xlabel("200-period", fontsize=17)
ax3.set_ylabel("Value", fontsize=17)
ax3.grid()
fig3.savefig('fig1.png')
plt.show()

In [ ]:
def IDID(df, t, c):
    df1 = copy.deepcopy(df)
    df1['time1'] = np.where(df1['time'] >= t, 1, 0)
    df1['treated'] = np.where(df1['AB'] == c, 1, 0)
    df1['did'] = df1['time1'] * df1['treated']
    timed = df1['time1']
    treatedd = df1['treated']
    didd = df1['did']
    y = df1['value']
    res = smf.ols(formula='y ~ didd + timed + treatedd', data=df1).fit()
    pv = res.pvalues[1]
    cofidid = res.params.tolist()
    return cofidid[1], pv

In [ ]:
listalldid = []
listpv = []

for i in x[1:]:
    alldid = IDID(df, i, 'A')
    listalldid.append(alldid[0])
    listpv.append(alldid[1])

In [ ]:
start_time = time.time()
listci = []
for i in np.linspace(3, 196, 194):
    print(i)
    ci = CausalImpact(yfinal, [0, int(i)], [int(i) + 1, 199],
    model_args={'fit_method': 'hmc'})
    listci.append(ci)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
z = 0
for ci in listci:
    string = ci.summary()
    print(string)
    z = z + 1
z = 0
for ci in listci:
    string = ci.summary(output='report')
    print(string)
    z = z + 1
for ci in listci:
    ci.plot()

In [ ]:
def custom_loss(y_true, y_pred):
    max1 = tf.math.maximum(y_true, y_pred)
    max2 = tf.reduce_sum(max1)
    min1 = tf.math.minimum(y_true, y_pred)
    min2 = tf.reduce_sum(min1)
    jaccard = (1 - (min2 / max2)) * 100
    return jaccard

In [ ]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        % find the end of this pattern
        end_ix = i + n_steps
        % check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
    % gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
listann = x[33:-15]
result_final = []
for j in listann:
    range_seed = [123]
    result_seed = []
    for i in range_seed:
        print(j, '---------------------------------------------------')
        best_ep, best_cicle, val_mae_best, list_result_fin = bi_lstm_iteration(int(j), i, listadd)
        print(list_result_fin)
        parameter = list_result_fin.index(min(list_result_fin)
        )
        min_best_loss = best_ep[parameter]
        mi_val_mae_best = val_mae_best[parameter]
        best_cicle_select = best_cicle[parameter] + 1
        list_cod_step = [8, 9, 10, 11, 12, 13, 14, 15, 16]
        best_step_select = list_cod_step[parameter]
        print(min(list_result_fin), min_best_loss,
            mi_val_mae_best, best_cicle_select,
            best_step_select, i, j)
        result_seed.append([min(list_result_fin),
            min_best_loss, mi_val_mae_best, best_cicle_select,
            best_step_select, i, j])
    best_i = []
    for i in result_seed:
        best_i.append(i[0])
    re = best_i.index(min(best_i))
    result_final.append(result_seed[re])

In [ ]:
def bi_lstm_iteration(treatment_period, seed, data):
    pre_period = [0, treatment_period]
    post_period = [treatment_period + 1, 199]
    list_steps = [8, 9, 10, 11, 12, 13, 14, 15, 16]
    best_epoch_value = []
    best_epoch_cicle = []
    list_val_mae = []
    lista_result_fin = []
    for step in list_steps:
        seed_values = seed
        os.environ['PYTHONHASHSEED'] = str(seed_values)
        random.seed(seed_values)
        tf.random.set_seed(seed_values)
        PaAn = pre_period[1]
        parametro_inizio = pre_period[0]
        raw_seq = data[0:treatment_period]
        n_steps = step
        X, y = split_sequence(raw_seq, n_steps)
        n_features = 1
        X = X.reshape((X.shape[0], X.shape[1], n_features))
        callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=40)
        model = Sequential()
        model.add(Bidirectional(LSTM(50, activation='LeakyReLU'), input_shape=(n_steps, n_features)))
        model.add(Dense(1))
        model.compile(optimizer='Adam', loss=custom_loss,
        metrics='mae')
        history = model.fit(X, y, epochs=100, validation_split
        =0.33, callbacks=[callback], verbose=0)
        list_val_loss = history.history['val_loss']
        list_val_loss_mae = history.history['val_mae']
        xmin = min(list_val_loss)
        yindex = list_val_loss.index(xmin)
        val_mae_val = list_val_loss_mae[yindex]
        list_val_mae.append(val_mae_val)
        best_epoch_cicle.append(yindex)
        best_epoch_value.append(xmin)
        os.environ['PYTHONHASHSEED'] = str(seed_values)
        random.seed(seed_values)
        tf.random.set_seed(seed_values)
        model1 = Sequential()
        model1.add(Bidirectional(LSTM(50, activation='LeakyReLU'), input_shape=(n_steps, n_features)))
        model1.add(Dense(1))
        model1.compile(optimizer='Adam', loss=custom_loss,
        metrics='mae')
        history1 = model1.fit(X, y, epochs=yindex+1,
        validation_split=0.33, verbose=0)
        yhat = model1.predict(X, verbose=1)
        prediction = []
        current_batch = np.array(yhat[:PaAn])[-n_steps:]
        current_batch = current_batch.reshape(1, n_steps,
        n_features)
        for i in range(10):
            current_pred = model1.predict(current_batch,
            verbose=0)[0]
            prediction.append(current_pred)
            current_batch = np.append(current_batch[:, 1:, :],
            [[current_pred]], axis=1)
        observed = listadd[PaAn:PaAn+10]
        resultfin = np.mean(np.array(observed) - np.array(
        prediction))
        listaresultfin.append(abs(resultfin))
    return best_epoch_value, best_epoch_cicle, list_val_mae, listaresultfin
